# Mediana dla obrazu kolorowego

Idea filtracji medianowej jest dość prosta dla obrazów w odcieniach szarości.
Dla obrazów kolorowych trudniej jest określić kryterium wg. którego szeregowane będą wartości, z których wyznaczana będzie mediana.

Jedną z możliwości wykonania filtracji medianowej dla obrazów kolorowych (na podstawie *The Image Processing Handbook*, J. Russ) jest wykorzystanie następującej definicji mediany:
``mediana to ten piksel z otoczenia, którego odległość do innych pikseli z otoczenia jest najmniejsza''.
Jako miarę odległości wykorzystujemy pierwiastek z sumy kwadratów różnic poszczególnych składowych R,G,B.
Zatem odległość między dwoma pikselami wyraża się wzorem:
\begin{equation}
dRGB = \sqrt{(R_1-R_2)^2+(G_1-G_2)^2+(B_1-B_2)^2}
\end{equation}

Warto zwrócić uwagę, że istnieje wiele możliwości zdefiniowania porównywania wielkości wektorowych (jeden piksel to wektor o trzech składowych).
Można zamiast odległości wykorzystać kąt albo połączyć oba parametry.
Ponadto istnieje możliwość dodania do wektora dodatkowych składowych - tak aby lepiej opisać piksel.

Celem zadania jest implementacja opisanego algorytmu.

1. Wczytaj obraz *lenaRGBSzum.png* (dostępny na git).
2. Zdefiniuj rozmiar okna.
3. Wykonaj pętle po pikselach, dla których okno jest zdefiniowane (pomiń brzeg obrazu).
4. Dla każdego piksela pobierz okno o właściwym rozmiarze.
5. Wykonaj pętle po oknie, wewnątrz której obliczona zostanie suma odległości.
    - Obliczanie różnicy: `window - window[rowWin, colWin]`.
    - Obliczanie kwadratów: `np.square`.
    - Obliczanie pierwiastka: `np.sqrt`.
    - Obliczanie sumy metodą `.sum`.
6. Po obliczeniu macierzy odległości wyznacz argument elementu minimalnego.
Wykorzystaj funkcję `np.argmin`.
Argument funkcji zostanie spłaszczony, jeśli ma więcej niż jeden wymiar.
Aby przekonwertować spłaszczony indeks na indeks macierzy wykorzystaj funkcję `np.unravel_index`.
7. Przypisz odpowiedni wektor wartości do piksela obrazu wynikowego.
8. Wyświetl obraz oryginalny i przefiltrowany.
9. Przeprowadź dwa eksperymenty - dla obrazu _lenaRGB_ oraz _lenaRGBszum_.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/'

fileNames = ["lenaRGB.png", "lenaRGBSzum.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

In [ ]:
def subplot_image(ax, image, title="", **kwargs):
    ax.set_title(title)
    ax.axis("off")
    ax.imshow(image, cmap=kwargs.get("cmap", "gray"))

def plot_image(image, title="", **kwargs):
    plt.title(title)
    plt.axis("off")
    plt.imshow(image, cmap=kwargs.get("cmap", "gray"))
    plt.show()

lena_szum = cv2.cvtColor(cv2.imread("lenaRGBSzum.png"), cv2.COLOR_BGR2RGB)
lena = cv2.cvtColor(cv2.imread("lenaRGB.png"), cv2.COLOR_BGR2RGB)

In [ ]:
class Kernel:
    def __init__(self, window):
        self.window = window.copy()
        self.H, self.W, self.C = self.window.shape

    def sub(self, color):
        window = self.window.copy().astype(np.float64)
        for chan in range(self.C):
            window[:, :, chan] -= color[chan]
        return window

    def median(self):
        self.window = self.window.astype(np.float64)
        dst_matrix = np.zeros((self.H, self.W))
        for h in range(self.H):
            for w in range(self.W):
                matrix = np.abs(np.square(self.sub(self.window[h, w])))
                dst = np.sqrt(np.sum(matrix, axis=2)).sum()
                dst_matrix[h, w] = dst
        return self.window[np.unravel_index(np.argmin(dst_matrix), (self.H, self.W))]


def color_median_conv(image, kernel_size):
    half = kernel_size // 2
    H, W, C = image.shape
    result = np.zeros((H - 2 * half, W - 2 * half, C))
    for row in range(half, H - half):
        for col in range(half, W - half):
            kernel = Kernel(image[row - half:row + half + 1, col - half:col + half + 1])
            result[row - half, col - half] = kernel.median()

    return result


def compare_images(
        left_img, 
        mid_img, 
        left_title="Original", 
        mid_title="", 
        **kwargs
    ):

    left_img = left_img.astype(np.int16)
    mid_img = mid_img.astype(np.int16)
    _, (left, mid) = plt.subplots(1, 2, figsize=(10, 5))
    subplot_image(left, left_img, left_title)
    subplot_image(mid, mid_img, mid_title)
    plt.show()


compare_images(lena_szum, color_median_conv(lena_szum, 3), mid_title="After median conv 3x3")
compare_images(lena_szum, color_median_conv(lena_szum, 5), mid_title="After median conv 5x5")
compare_images(lena_szum, color_median_conv(lena_szum, 7), mid_title="After median conv 7x7")

In [ ]:
compare_images(lena, color_median_conv(lena, 3), mid_title="After median conv 3x3")
compare_images(lena, color_median_conv(lena, 5), mid_title="After median conv 5x5")
compare_images(lena, color_median_conv(lena, 7), mid_title="After median conv 7x7")